### 1. Clipping Video Per 3 Detik

In [ ]:
import os
import cv2

def split_and_filter(input_path, output_folder, label, clip_duration=3):
    os.makedirs(output_folder, exist_ok=True)

    cap = cv2.VideoCapture(input_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames_per_clip = int(fps * clip_duration)

    clip_index = 1
    frame_index = 0
    out = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Setiap mulai 1 clip baru
        if frame_index % frames_per_clip == 0:
            # Tutup writer sebelumnya
            if out is not None:
                out.release()

            # Tentukan apakah clip ini disimpan atau tidak
            save_clip = True

            if label == "cheating":
                # Clip ganjil = disimpan
                # Clip genap  = normal = dibuang
                if clip_index % 2 == 0:
                    save_clip = False

            if save_clip:
                clip_name = f"{os.path.splitext(os.path.basename(input_path))[0]}_clip{clip_index:03d}.mp4"
                output_path = os.path.join(output_folder, clip_name)

                fourcc = cv2.VideoWriter_fourcc(*"mp4v")
                out = cv2.VideoWriter(output_path, fourcc, fps, (frame.shape[1], frame.shape[0]))
            else:
                out = None

            clip_index += 1

        # Kalau clip ini disimpan, tulis frame
        if out is not None:
            out.write(frame)

        frame_index += 1

    if out is not None:
        out.release()
    cap.release()



def process_dataset(base_path="dataset", clip_duration=3):
    for label in os.listdir(base_path):
        label_path = os.path.join(base_path, label)
        if not os.path.isdir(label_path):
            continue

        for angle in os.listdir(label_path):
            angle_path = os.path.join(label_path, angle)
            if not os.path.isdir(angle_path):
                continue

            output_path = os.path.join("dataset_clips", label, angle)
            os.makedirs(output_path, exist_ok=True)

            for filename in os.listdir(angle_path):
                if filename.endswith((".mp4", ".avi", ".mov")):
                    input_path = os.path.join(angle_path, filename)
                    split_and_filter(input_path, output_path, label, clip_duration)

                    print(f"✔ Proses selesai: {input_path}")


# Jalankan
process_dataset("/run/media/han/HDD RAIHAN/Dataset Cheating Comvis", clip_duration=3)

### 2. Membagi Dataset ke Folder train, val, test  

In [ ]:
import os
import shutil
import random
from pathlib import Path

# ==== KONFIGURASI ====
SRC_DIR = Path("/home/han/Documents/Kuliah/S5/Comvis/Dataset Video/dataset_clips")
OUT_DIR = Path("/home/han/Documents/Kuliah/S5/Comvis/Dataset Video/dataset_ready_klasifikasi")

SPLIT = {
    "train": 0.7,
    "val": 0.15,
    "test": 0.15
}

VIDEO_EXT = [".mp4", ".avi", ".mov", ".mkv"]

# ==== Buat folder output ====
for split in SPLIT:
    (OUT_DIR / split / "cheating").mkdir(parents=True, exist_ok=True)
    (OUT_DIR / split / "not_cheating").mkdir(parents=True, exist_ok=True)

def list_all_clips(class_name):
    """Mengambil semua clip dari folder Cheating/Not Cheating termasuk subfoldernya."""
    clips = []
    class_path = SRC_DIR / class_name

    for root, _, files in os.walk(class_path):
        for f in files:
            if any(f.lower().endswith(ext) for ext in VIDEO_EXT):
                clips.append(Path(root) / f)

    return clips

# ==== Ambil semua clip ====
cheat_clips = list_all_clips("cheating")
not_cheat_clips = list_all_clips("not_cheating")

print("[INFO] Total Cheating:", len(cheat_clips))
print("[INFO] Total Not Cheating:", len(not_cheat_clips))

def split_dataset(files):
    """Split file list ke train/val/test."""
    random.shuffle(files)
    n = len(files)
    t = int(SPLIT["train"] * n)
    v = int(SPLIT["val"] * n)

    return {
        "train": files[:t],
        "val": files[t:t+v],
        "test": files[t+v:]
    }

cheat_split = split_dataset(cheat_clips)
not_cheat_split = split_dataset(not_cheat_clips)

# ==== COPY DENGAN NAMA BARU ====
def copy_with_new_name(files, split, class_name):
    for idx, src in enumerate(files):
        new_name = f"{class_name.lower().replace(' ', '_')}_{idx:05d}.mp4"
        dst = OUT_DIR / split / class_name / new_name
        shutil.copy2(src, dst)

# Cheating
for split in SPLIT:
    copy_with_new_name(cheat_split[split], split, "cheating")

# Not Cheating
for split in SPLIT:
    copy_with_new_name(not_cheat_split[split], split, "not_cheating")
print("=== Dataset Berhasil Dibuat! ===")
print("Lokasi:", OUT_DIR)

### 3. Crop Video Menggunakan YOLO 

In [ ]:
import cv2
import os
import numpy as np
from ultralytics import YOLO
from tqdm import tqdm

# --- KONFIGURASI UTAMA ---
INPUT_ROOT = "/home/han/Documents/Kuliah/S5/Comvis/Dataset Video/dataset_ready_klasifikasi"          # Folder Input (Dataset Asli)
OUTPUT_ROOT = "/home/han/Documents/Kuliah/S5/Comvis/Dataset Video/dataset_ready_smoothing" # Folder Output
TARGET_SIZE = 112                             # Ukuran input R3D

# --- KONFIGURASI SMOOTHING ---
# Nilai alpha (0.0 sampai 1.0)
# 0.1 = Sangat smooth (lambat, seperti kamera cinematic)
# 0.9 = Sangat responsif (cepat, tapi masih ada sedikit getar)
# 0.6 - 0.7 = Angka ideal untuk kasus Anda.
SMOOTHING_FACTOR = 0.7 

# Load YOLO
model = YOLO("yolov8n.pt") 

# --- FUNGSI 1: CROP & PAD (Agar Gambar Tidak Gepeng) ---
def crop_padded_square(frame, box, target_size):
    h, w, _ = frame.shape
    x1, y1, x2, y2 = map(int, box)
    
    # Safety Check: Pastikan koordinat ada di dalam gambar
    x1 = max(0, x1); y1 = max(0, y1)
    x2 = min(w, x2); y2 = min(h, y2)
    
    crop = frame[y1:y2, x1:x2]
    
    if crop.size == 0:
        return np.zeros((target_size, target_size, 3), dtype=np.uint8)

    # Hitung Padding agar jadi KOTAK (Square)
    ch, cw, _ = crop.shape
    delta_w = max(ch - cw, 0)
    delta_h = max(cw - ch, 0)
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)

    # Tambah Border Hitam
    color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(crop, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)

    # Resize ke 112x112
    final_im = cv2.resize(new_im, (target_size, target_size))
    return final_im

# --- FUNGSI 2: PROSES VIDEO DENGAN SMOOTHING ---
def process_video_smoothed(input_path, output_dir, filename_base):
    cap = cv2.VideoCapture(input_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    if fps == 0: fps = 30
    
    writers = {}
    
    # MEMORY: Menyimpan posisi kotak frame sebelumnya per ID
    # Format: { track_id: [x1, y1, x2, y2] }
    previous_boxes = {} 

    while True:
        ret, frame = cap.read()
        if not ret: break
        
        # Tracking YOLO
        results = model.track(frame, classes=[0], persist=True, verbose=False, tracker="bytetrack.yaml")
        
        if results[0].boxes.id is not None:
            boxes = results[0].boxes.xyxy.cpu().numpy()
            track_ids = results[0].boxes.id.cpu().numpy().astype(int)
            
            for box, track_id in zip(boxes, track_ids):
                current_box = box # [x1, y1, x2, y2] float
                
                # --- LOGIKA SMOOTHING (EMA) ---
                if track_id in previous_boxes:
                    prev_box = previous_boxes[track_id]
                    # Rumus: BoxBaru = (0.7 * BoxSekarang) + (0.3 * BoxLama)
                    smoothed_box = (SMOOTHING_FACTOR * current_box) + ((1 - SMOOTHING_FACTOR) * prev_box)
                else:
                    # Jika ID baru muncul, langsung pakai posisi asli
                    smoothed_box = current_box
                
                # Simpan posisi smooth ini untuk frame selanjutnya
                previous_boxes[track_id] = smoothed_box
                
                # Konversi ke integer untuk cropping
                final_box_int = smoothed_box.astype(int)
                # ------------------------------

                # Siapkan Video Writer
                if track_id not in writers:
                    out_name = f"{filename_base}_id{track_id}.mp4"
                    out_path = os.path.join(output_dir, out_name)
                    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
                    writers[track_id] = cv2.VideoWriter(out_path, fourcc, fps, (TARGET_SIZE, TARGET_SIZE))
                
                # Crop dengan Padding (Square)
                final_frame = crop_padded_square(frame, final_box_int, TARGET_SIZE)
                
                # Simpan frame
                writers[track_id].write(final_frame)

    cap.release()
    for w in writers.values():
        w.release()

# --- MAIN LOOP ---
print("🚀 Mulai Memproses Dataset (Mode SMOOTHING)...")

for split in ["train", "val"]:
    for cls in ["cheating", "not_cheating"]:
        
        input_dir = os.path.join(INPUT_ROOT, split, cls)
        output_dir = os.path.join(OUTPUT_ROOT, split, cls)
        
        if not os.path.exists(input_dir): 
            continue
            
        os.makedirs(output_dir, exist_ok=True)
        videos = [f for f in os.listdir(input_dir) if f.endswith(".mp4")]
        
        print(f"\n📂 Processing {split}/{cls} ({len(videos)} videos)...")
        
        for vid_name in tqdm(videos):
            input_path = os.path.join(input_dir, vid_name)
            filename_base = os.path.splitext(vid_name)[0]
            
            process_video_smoothed(input_path, output_dir, filename_base)

print("\n✅ Selesai! Video bounding box kini stabil dan mulus.")